# N reinas
### Consigna
Encontrar todas las soluciones posibles que ubican en un tablero de ajedrez de NxN, N reinas sin que se coman entre sí. Las reinas se pueden mover sin límite por fila, columna o en diagonal, es decir, no puede haber dos reinas en la misma fila, columna o diagonal.

### Algoritmos utilizados
* Fuerza bruta
* Backtracking

In [1]:
from pandas import *
from IPython.display import HTML

def color_styling(val):
    color = 'red' if val == 0 else 'black'
    return 'color: %s' % color

class Tablero:
    def __init__(self, N):
        self.N = N
        self.reinas = set()
        self.cantidad_reinas = 0
        self.posiciones_validas = set()
        for i in range (1,N+1):
            for j in range(1, N+1):
                self.posiciones_validas.add((i, j))
        self.tablero_df = DataFrame()
    
    def es_factible(self, posicion):
        return posicion in self.posiciones_validas
    
    def agregar_posicion(self, posicion):
        x = posicion[0]
        y = posicion[1]
        # elimina filas y columnas.
        for i in range(1, self.N+1):
            # .discard() elimina el elemento si existe
            self.posiciones_validas.discard((x, i))
            self.posiciones_validas.discard((i, y))
        # elimina diagonales hacia la derecha
        for i in range(1, self.N + 1 - x):
            self.posiciones_validas.discard((x + i, y + i))
            self.posiciones_validas.discard((x + i, y - i))
        # elimina diagonales hacia la izquierda
        for i in range(1, x):
            self.posiciones_validas.discard((x - i, y + i))
            self.posiciones_validas.discard((x - i, y - i))
        self.reinas.add(posicion)
        self.cantidad_reinas += 1
    
    def actualizar_representacion(self):
        tablero = []
        for i in range(1, self.N+1):
            fila = []
            for j in range(1, self.N+1):
                if ((i, j) in self.reinas):
                    fila.append('R')
                elif ((i, j) in self.posiciones_validas):
                    fila.append('_')
                else:
                    fila.append('x')
            tablero.append(fila)
        self.tablero_df = DataFrame(tablero, columns = [i for i in range(1, self.N+1)], index = [i for i in range(1, self.N+1)])
        
    def __str__(self):
        self.actualizar_representacion()
        return self.tablero_df.__str__()
        
    def copy(self):
        t = Tablero(self.N)
        t.posiciones_validas = self.posiciones_validas.copy()
        t.tablero_df = self.tablero_df.copy()
        t.cantidad_reinas = self.cantidad_reinas
        t.reinas = self.reinas.copy()
        return t

In [2]:
t = Tablero(5)
t.agregar_posicion((1,2))
t.actualizar_representacion()
print(t.tablero_df)

   1  2  3  4  5
1  x  R  x  x  x
2  x  x  x  _  _
3  _  x  _  x  _
4  _  x  _  _  x
5  _  x  _  _  _


## Fuerza bruta
Vamos a probar todas las combinaciones posibles

In [3]:
def fuerza_bruta(tablero):
    soluciones = []
    _fuerza_bruta(tablero, soluciones, 1, 0)
    return soluciones

def _fuerza_bruta(tablero, soluciones, fila, columna):
    # Caso base
    if columna > tablero.N:
        return
    if (tablero.es_factible((fila, columna))):
        tablero.agregar_posicion((fila,columna))
        if (tablero.cantidad_reinas == tablero.N):
            soluciones.append(tablero.copy())
    for i in range(1, tablero.N + 1):
        _fuerza_bruta(tablero.copy(), soluciones, i, columna + 1)

In [4]:
from time import time
t = Tablero(6)
initial_time = time()
soluciones = fuerza_bruta(t)
print("Con fuerza bruta tardó: ", time() - initial_time, " segundos.")
print("Cantidad de soluciones: ",len(soluciones))
for i in range(len(soluciones)):
    print('\n\n', soluciones[i])

Con fuerza bruta tardó:  109.45391297340393  segundos.
Cantidad de soluciones:  4


    1  2  3  4  5  6
1  x  x  x  R  x  x
2  R  x  x  x  x  x
3  x  x  x  x  R  x
4  x  R  x  x  x  x
5  x  x  x  x  x  R
6  x  x  R  x  x  x


    1  2  3  4  5  6
1  x  x  x  x  R  x
2  x  x  R  x  x  x
3  R  x  x  x  x  x
4  x  x  x  x  x  R
5  x  x  x  R  x  x
6  x  R  x  x  x  x


    1  2  3  4  5  6
1  x  R  x  x  x  x
2  x  x  x  R  x  x
3  x  x  x  x  x  R
4  R  x  x  x  x  x
5  x  x  R  x  x  x
6  x  x  x  x  R  x


    1  2  3  4  5  6
1  x  x  R  x  x  x
2  x  x  x  x  x  R
3  x  R  x  x  x  x
4  x  x  x  x  R  x
5  R  x  x  x  x  x
6  x  x  x  R  x  x


## Backtracking
Ahora podemos los caminos que ya sabemos que no pueden funcionar

In [5]:
def backtracking(tablero):
    soluciones = []
    _backtracking(tablero, soluciones, 1, 0)
    return soluciones

def _backtracking(tablero, soluciones, fila, columna):
    # Caso base
    if columna > tablero.N:
        return
    # Si la columna es 0 llamo a BT para todas las filas de la primer columna
    if columna == 0:
        for i in range(1, tablero.N + 1):
            _backtracking(tablero.copy(), soluciones, i, columna + 1)
    if (tablero.es_factible((fila, columna))):
        tablero.agregar_posicion((fila,columna))
        if (tablero.cantidad_reinas == tablero.N):
            soluciones.append(tablero.copy())
        for i in range(1, tablero.N + 1):
            _backtracking(tablero.copy(), soluciones, i, columna + 1)

In [6]:
t = Tablero(6)
initial_time = time()
soluciones = backtracking(t)
print("Con backtracking tardó: ", time() - initial_time, " segundos.")
print("Cantidad de soluciones: ",len(soluciones))
for i in range(len(soluciones)):
    print('\n\n', soluciones[i])

Con backtracking tardó:  0.2924158573150635  segundos.
Cantidad de soluciones:  4


    1  2  3  4  5  6
1  x  x  x  R  x  x
2  R  x  x  x  x  x
3  x  x  x  x  R  x
4  x  R  x  x  x  x
5  x  x  x  x  x  R
6  x  x  R  x  x  x


    1  2  3  4  5  6
1  x  x  x  x  R  x
2  x  x  R  x  x  x
3  R  x  x  x  x  x
4  x  x  x  x  x  R
5  x  x  x  R  x  x
6  x  R  x  x  x  x


    1  2  3  4  5  6
1  x  R  x  x  x  x
2  x  x  x  R  x  x
3  x  x  x  x  x  R
4  R  x  x  x  x  x
5  x  x  R  x  x  x
6  x  x  x  x  R  x


    1  2  3  4  5  6
1  x  x  R  x  x  x
2  x  x  x  x  x  R
3  x  R  x  x  x  x
4  x  x  x  x  R  x
5  R  x  x  x  x  x
6  x  x  x  R  x  x
